# Introduction to Letta
This lab will go over: 
1. Creating an agent with Letta
2. Understand Letta agent state (messages, memories, tools)
3. Understanding core and archival memory
4. Building agentic RAG with Letta

## Section 0: Setup a Letta client 

In [1]:
from letta import create_client 

client = create_client() 

Initializing database...


In [2]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002")) 

## Section 1: Creating a simple agent with memory 
Letta allows you to create persistent LLM agents that have memory. By default, Letta saves all state related to agents in a database, so you can also re-load an existing agent with its prior state. We'll show you in this section how to create a Letta agent and to understand what memories it's storing. 


### Creating an agent 

In [3]:
agent_name = "simple_agent"

In [4]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [5]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)
response

LettaResponse(messages=[InternalMonologue(id='message-896802ce-b3b9-444b-abd9-b0d20fd49681', date=datetime.datetime(2024, 11, 7, 4, 14, 59, 675860, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User has logged in, greeting them back!'), FunctionCallMessage(id='message-896802ce-b3b9-444b-abd9-b0d20fd49681', date=datetime.datetime(2024, 11, 7, 4, 14, 59, 675860, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Hey there! 👋 How\'s it going?"\n}', function_call_id='call_b6fl10gRrCpgWXLkpx50jc3r')), FunctionReturn(id='message-87b61f26-c2ed-4d78-ad40-dbf7321d77e3', date=datetime.datetime(2024, 11, 7, 4, 14, 59, 677137, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:14:59 PM PST-0800"\n}', status='success', function_call_id='call_b6fl10gRrCpgWXLkpx50jc3r')], usage=LettaUsageStatistics(completion_tokens=38, prompt_tokens=2145, total_tokens=2183, step_count=1))

Note that Letta agents will generate an *internal_monologue* that explains its actions. You can use this monoloque to understand why agents are behaving as they are. 

Second, Letta agents also use tools to communicate, so messages are sent back by calling  a `send_message` tool. This makes it easy to allow agent to communicate over different mediums (e.g. text), and also allows the agent to distinguish betweeh that is and isn't send to the end user. 

### Understanding agent state 
Letta agents are *stateful* and are defined by: 
* The system prompt defining the agent's behavior (read-only)
* The set of *tools* they have access to 
* Their memory (core, archival, & recall)

In [6]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [7]:
agent_state.tools

['send_message',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace']

### Viewing an agent's memory

In [8]:
memory = client.get_core_memory(agent_state.id)

In [9]:
memory

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-e018b490-f3c2-4fb4-95fe-750cbe140a0b'), 'human': Block(value='My name is Sarah', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d7d64a4f-465b-45ca-89e6-763fe161c2b6')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [10]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [11]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=9)

In [12]:
client.get_messages(agent_state.id)

[Message(id='message-87b61f26-c2ed-4d78-ad40-dbf7321d77e3', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:14:59 PM PST-0800"\n}', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-33c66d6d-3b2b-4a45-aeb3-7e08344bdef9', model='gpt-4o-mini', name='send_message', created_at=datetime.datetime(2024, 11, 7, 4, 14, 59, 677137), tool_calls=None, tool_call_id='call_b6fl10gRrCpgWXLkpx50jc3r'),
 Message(id='message-896802ce-b3b9-444b-abd9-b0d20fd49681', role=<MessageRole.assistant: 'assistant'>, text='User has logged in, greeting them back!', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-33c66d6d-3b2b-4a45-aeb3-7e08344bdef9', model='gpt-4o-mini', name=None, created_at=datetime.datetime(2024, 11, 7, 4, 14, 59, 675860), tool_calls=[ToolCall(id='call_b6fl10gRrCpgWXLkpx50jc3r', type='function', function=ToolCallFunction(name='send_message', arguments='{\n  "message": "Hey there! 👋 How\'s it going?"

## Section 2: Understanding core memory 
Core memory is memory that is stored *in-context* - so every LLM call, core memory is included. What's unique about Letta is that this core memory is editable via tools by the agent itself. Lets see how the agent can adapt its memory to new information.

### Memories about the human 
The `human` section of `ChatMemory` is used to remember information about the human in the conversation. As the agent learns new information about the human, it can update this part of memory to improve personalization. 

In [13]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Bob", 
    role = "user"
) 
response

/Users/sarahwooders/repos/letta/letta/helpers/tool_rule_solver.py:70: UserWarning: User provided tool rules and execution state resolved to no more possible tool calls.
  warnings.warn(message)


LettaResponse(messages=[InternalMonologue(id='message-c01674a2-7b18-4264-a422-9f03e340c60b', date=datetime.datetime(2024, 11, 7, 4, 16, 1, 339591, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Updating user name in memory to Bob.'), FunctionCallMessage(id='message-c01674a2-7b18-4264-a422-9f03e340c60b', date=datetime.datetime(2024, 11, 7, 4, 16, 1, 339591, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "Sarah",\n  "new_content": "Bob",\n  "request_heartbeat": true\n}', function_call_id='call_QWVubWm1EyreprZ448b7O9BK')), FunctionReturn(id='message-1ec685c0-d626-415d-a0d5-a380c481167e', date=datetime.datetime(2024, 11, 7, 4, 16, 1, 340857, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:16:01 PM PST-0800"\n}', status='success', function_call_id='call_QWVubWm1EyreprZ448b7O9BK'), InternalMonologue(id='message-1917419f-e6d4-4783-81eb-7aff2db0dc2e', date=datetime.datetime(2024, 11, 7, 4, 16, 4, 777960, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Just updated the name. Time to engage Bob!'), FunctionCallMessage(id='message-1917419f-e6d4-4783-81eb-7aff2db0dc2e', date=datetime.datetime(2024, 11, 7, 4, 16, 4, 777960, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it, Bob! Nice to officially meet you! 😄 What’s on your mind today?"\n}', function_call_id='call_WKCrcPq1LVuJE7xmNjNnrEog')), FunctionReturn(id='message-b40a9738-8870-4fdb-b737-c451a0e8f357', date=datetime.datetime(2024, 11, 7, 4, 16, 4, 780317, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:16:04 PM PST-0800"\n}', status='success', function_call_id='call_WKCrcPq1LVuJE7xmNjNnrEog')], usage=LettaUsageStatistics(completion_tokens=93, prompt_tokens=4712, total_tokens=4805, step_count=2))

In [14]:
client.get_core_memory(agent_state.id)

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-e018b490-f3c2-4fb4-95fe-750cbe140a0b'), 'human': Block(value='My name is Bob', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d7d64a4f-465b-45ca-89e6-763fe161c2b6')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

### Memories about the agent
The agent also records information about itself and how it behaves in the `persona` section of memory. This is important for ensuring a consistent persona over time (e.g. not making inconsistent claims, such as liking ice cream one day and hating it another). Unlike the `system_prompt`, the `persona` is editable - this means that it can be used to incoporate feedback to learn and improve its persona over time. 

In [15]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
response

LettaResponse(messages=[InternalMonologue(id='message-be1d57a6-50a2-4037-af90-1cddc0e8077b', date=datetime.datetime(2024, 11, 7, 4, 29, 12, 914967, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User prefers no emojis in communication. Updating memory accordingly.'), FunctionCallMessage(id='message-be1d57a6-50a2-4037-af90-1cddc0e8077b', date=datetime.datetime(2024, 11, 7, 4, 29, 12, 914967, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "likes emojis",\n  "new_content": "doesn\'t like emojis",\n  "request_heartbeat": true\n}', function_call_id='call_zNDfyPm2FAecwVtXxnWDc4Vu')), FunctionReturn(id='message-35fe066e-e6bc-4957-adf2-85aa9a2d1e87', date=datetime.datetime(2024, 11, 7, 4, 29, 12, 917213, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "Failed",\n  "message": "Error calling function core_memory_replace: Old content \'likes emojis\' not found in memory block \'human\'",\n  "time": "2024-11-06 08:29:12 PM PST-0800"\n}', status='error', function_call_id='call_zNDfyPm2FAecwVtXxnWDc4Vu'), InternalMonologue(id='message-98a6c2f1-da48-47d7-9af0-f650be7fd4cf', date=datetime.datetime(2024, 11, 7, 4, 29, 14, 133464, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User dislikes emojis. Adding this to memory.'), FunctionCallMessage(id='message-98a6c2f1-da48-47d7-9af0-f650be7fd4cf', date=datetime.datetime(2024, 11, 7, 4, 29, 14, 133464, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_append', arguments='{\n  "label": "human",\n  "content": "dislikes emojis",\n  "request_heartbeat": true\n}', function_call_id='call_mRoQbWfAOokv269dlbKpyg6g')), FunctionReturn(id='message-4ce1f1a1-9fc5-4b6c-9ad5-84a46b0153ca', date=datetime.datetime(2024, 11, 7, 4, 29, 14, 134502, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:29:14 PM PST-0800"\n}', status='success', function_call_id='call_mRoQbWfAOokv269dlbKpyg6g'), InternalMonologue(id='message-0bbdb6d6-2f4b-45ea-9452-f6466aae7ac5', date=datetime.datetime(2024, 11, 7, 4, 29, 18, 402937, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Failed to update memory earlier, but now added a dislike for emojis. Ready to communicate accordingly!'), FunctionCallMessage(id='message-0bbdb6d6-2f4b-45ea-9452-f6466aae7ac5', date=datetime.datetime(2024, 11, 7, 4, 29, 18, 402937, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Understood, Bob! I won\'t use emojis anymore. What would you like to talk about?"\n}', function_call_id='call_8vqVfG44CPsG1SkdF3SByQGi')), FunctionReturn(id='message-cdb126d5-1c92-42f6-a3e1-a6676671f781', date=datetime.datetime(2024, 11, 7, 4, 29, 18, 404241, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:29:18 PM PST-0800"\n}', status='success', function_call_id='call_8vqVfG44CPsG1SkdF3SByQGi')], usage=LettaUsageStatistics(completion_tokens=149, prompt_tokens=8259, total_tokens=8408, step_count=3))

In [16]:
client.get_core_memory(agent_state.id).get_block('persona')

Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-e018b490-f3c2-4fb4-95fe-750cbe140a0b')

## Section 3: Understanding archival memory
Letta agents store long term memories in *archival memory*, which persists data into an external database. This allows agents additional space to write information outside of its context window (e.g. with core memory), which is limited in size. 

In [17]:
client.get_archival_memory(agent_state.id)

[]

In [18]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

Agents themselves can write to their archival memory when they learn information they think should be placed in long term storage. You can also directly suggest that the agent store information in archival. 

In [19]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
response

LettaResponse(messages=[InternalMonologue(id='message-5a2bb25e-78e8-4c10-87fc-2cb27d872d1d', date=datetime.datetime(2024, 11, 7, 4, 29, 20, 652683, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User Bob loves cats. Saving this in archival memory for future reference.'), FunctionCallMessage(id='message-5a2bb25e-78e8-4c10-87fc-2cb27d872d1d', date=datetime.datetime(2024, 11, 7, 4, 29, 20, 652683, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='archival_memory_insert', arguments='{\n  "content": "Bob loves cats",\n  "request_heartbeat": true\n}', function_call_id='call_dzxwS4o30WgbkXx0gbLssj9T')), FunctionReturn(id='message-2b9633aa-91ac-4c7e-861c-ce71056e7b85', date=datetime.datetime(2024, 11, 7, 4, 29, 21, 338360, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:29:21 PM PST-0800"\n}', status='success', function_call_id='call_dzxwS4o30WgbkXx0gbLssj9T'), InternalMonologue(id='message-e7816a60-8fc2-4de9-ab96-1cb73de943a7', date=datetime.datetime(2024, 11, 7, 4, 29, 24, 85675, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Successfully saved Bob's love for cats. Now ready for the next conversation!"), FunctionCallMessage(id='message-e7816a60-8fc2-4de9-ab96-1cb73de943a7', date=datetime.datetime(2024, 11, 7, 4, 29, 24, 85675, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got that saved, Bob! What else do you want to share or chat about?"\n}', function_call_id='call_b7YYrV68VRbgLizChsYjLkSc')), FunctionReturn(id='message-1bd009fc-0e84-4522-a27e-76b75ac848ff', date=datetime.datetime(2024, 11, 7, 4, 29, 24, 86646, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:29:24 PM PST-0800"\n}', status='success', function_call_id='call_b7YYrV68VRbgLizChsYjLkSc')], usage=LettaUsageStatistics(completion_tokens=94, prompt_tokens=6306, total_tokens=6400, step_count=2))

In [20]:
client.get_archival_memory(agent_state.id)[0].text

'Bob loves cats'

You can also directly insert into archival memory from the client. 

In [21]:
client.insert_archival_memory(
    agent_state.id, 
    "Bob's loves boston terriers"
)

[Passage(user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-33c66d6d-3b2b-4a45-aeb3-7e08344bdef9', source_id=None, file_id=None, metadata_={}, id='passage-0c6ba187-0ce8-4c5f-8dfb-fde5c567a48d', text="Bob's loves boston terriers", embedding=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='https://api.openai.com/v1', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), created_at=datetime.datetime(2024, 11, 6, 20, 29, 24))]

Now lets see how the agent uses its archival memory:

In [22]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
response

LettaResponse(messages=[InternalMonologue(id='message-291f7c38-77a2-4a1c-a6da-0674ebd909ac', date=datetime.datetime(2024, 11, 7, 4, 29, 28, 945422, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Looking for information on Bob's favorite animals in archival memory."), FunctionCallMessage(id='message-291f7c38-77a2-4a1c-a6da-0674ebd909ac', date=datetime.datetime(2024, 11, 7, 4, 29, 28, 945422, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='archival_memory_search', arguments='{\n  "query": "Bob loves cats",\n  "page": 0,\n  "request_heartbeat": true\n}', function_call_id='call_3ZYtBW1acTC1y2erHiMsrkyV')), FunctionReturn(id='message-97167b78-5813-45ce-9b19-00615619ff43', date=datetime.datetime(2024, 11, 7, 4, 29, 29, 346109, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "Showing 2 of 2 results (page 0/0): [\\n  \\"timestamp: 2024-11-06 08:29:29 PM PST-0800, memory: Bob loves cats\\",\\n  \\"timestamp: 2024-11-06 08:29:29 PM PST-0800, memory: Bob\'s loves boston terriers\\"\\n]",\n  "time": "2024-11-06 08:29:29 PM PST-0800"\n}', status='success', function_call_id='call_3ZYtBW1acTC1y2erHiMsrkyV'), InternalMonologue(id='message-37acba7a-e262-46f4-aa0d-c5db369d896a', date=datetime.datetime(2024, 11, 7, 4, 29, 31, 410686, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Found information on Bob's favorite animals. Sending it back to user."), FunctionCallMessage(id='message-37acba7a-e262-46f4-aa0d-c5db369d896a', date=datetime.datetime(2024, 11, 7, 4, 29, 31, 410686, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "You like cats and also Boston Terriers! What a great taste in pets, Bob! 🐱🐶"\n}', function_call_id='call_RyiWvh1h7KOxQbqibSZDx5c5')), FunctionReturn(id='message-c19bd9f5-7233-4df6-b420-48c49d73a60d', date=datetime.datetime(2024, 11, 7, 4, 29, 31, 412319, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:29:31 PM PST-0800"\n}', status='success', function_call_id='call_RyiWvh1h7KOxQbqibSZDx5c5')], usage=LettaUsageStatistics(completion_tokens=100, prompt_tokens=6998, total_tokens=7098, step_count=2))